In [1]:
import cobra
import os
import pandas
import glob

In [2]:
model_dict = dict()
path = "/home/mac9jc/paradigm/models/"
os.chdir(path)
for filename in glob.glob(os.path.join(path, 'gf_*.xml')):
    key = filename.split('/')[len(filename.split('/'))-1]
    key = key[:-4]
    key = key[3:]
    model_dict[key] = cobra.io.read_sbml_model(filename)

In [3]:
blocked = dict()
unconnected_upstream = dict()
both = dict()

for species, model in model_dict.items():
    blocked_down_rxns = list()
    blocked_up_rxns = list()
    both_rxns = list()
    model_reactions = [r.id for r in model.reactions]
    for x in model_reactions:
        rxn = model.reactions.get_by_id(x)
        rxn_products = rxn.products
        rxn_reactants = rxn.reactants
        temp_dict = dict()
        for met in rxn_products:
            temp_dict[met] = [r.id for r in met.reactions if met in r.reactants]
        if len([met.id for met in temp_dict.keys() if len(temp_dict[met])<1])>1:
            blocked_down_rxns.append(x)
        temp_dict = dict()
        for met in rxn_reactants:
            temp_dict[met] = [r.id for r in met.reactions if met in r.products]
        if len([met.id for met in temp_dict.keys() if len(temp_dict[met])<1])>1:
            blocked_up_rxns.append(x)
        if x in blocked_up_rxns and x in blocked_down_rxns:
            both_rxns.append(x)
        
        #if x in both_rxns or x in blocked_up_rxns or x in blocked_down_rxns: print({species: x})
        
    blocked[species] = blocked_down_rxns
    unconnected_upstream[species] = blocked_up_rxns
    both[species] = both_rxns
        
blocked = pandas.DataFrame.from_dict(blocked, orient='index')
unconnected_upstream = pandas.DataFrame.from_dict(unconnected_upstream, orient='index')
both = pandas.DataFrame.from_dict(both, orient='index')

In [6]:
unconnected_upstream.head()

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
no_ortho_EcuniculiEC3,PANTS,CERS226,CERS126,ALATA_D2,DIPS,HBZOPT6,FEDCabc,AGPAT120,L_LACD,ENLIPIDAt1exc,...,None,None,None,None,None,None,None,None,None,None
no_ortho_TevansiSTIB805,2AGPGAT141,HEMEOSm,CD2t4,2AGPGAT161,AHSERL3,2AGPEAT141,2HP6MPMOm,FTHFLmi,LYSTRSm,METTRSm,...,None,None,None,None,None,None,None,None,None,None
PfalciparumTG01,HEMEOSm,ILETRSm,THRTRSm,VALTRSm,CDGGGPP3,CD2t4,ASNTRSm,HISTRSm,PHETRSm,AGPAT190,...,None,None,None,None,None,None,None,None,None,None
no_ortho_TgondiiARI,2AGPGAT141,THRTRSm,VALTRSm,MEGLYXYLAMH,DHPS2,HISTRSm,PHETRSm,LIDOAMH,2AGPEAT141,DHORD2,...,None,None,None,None,None,None,None,None,None,None
no_ortho_TcruziCLBrenerNon-Esmeraldo-like,FAS141ACPm,FAS181ACPm,HEMEOSm,UPLA4FNT,FAS182ACPm,NALN6,AGAIAGT,2HP6MPMOm,FAS80ACPm_L,FTHFLmi,...,None,None,None,None,None,None,None,None,None,None


In [11]:
path = "/home/mac9jc/paradigm/data/results"
os.chdir(path)

# with pandas.ExcelWriter('supplemental_table_blocked_and_unconnected_reactions.xlsx') as writer:  
#     blocked.to_excel(writer, sheet_name='blocked')
#     unconnected_upstream.to_excel(writer, sheet_name='unconnected_upstream')
#     both.to_excel(writer, sheet_name='both')

blocked.to_csv("supplemental_table_blocked_reactions.csv")
unconnected_upstream.to_csv("supplemental_table_unconnected_reactions.csv")
both.to_csv("supplemental_table_blocked_and_unconnected_reactions.csv")
